<a href="https://colab.research.google.com/github/slala2121/Triplet-net-keras/blob/COS597D/deep_metric_learning_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code adapted from:
https://github.com/KinWaiCheuk/Triplet-net-keras

Other relevant links:

scratch classification network from https://keras.io/examples/cifar10_resnet/

In [8]:
!pip install --upgrade tensorflow
!pip install tensorflow-addons


Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.0.0)


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import time
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import random
from itertools import permutations
from PIL import Image
import cv2

import tensorflow as tf

from tensorflow import keras

import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.optimizers as optimizers

from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import binary_crossentropy

import tensorflow_addons as tfa


source_path=os.path.join('drive','My Drive', 'Colab Notebooks')


In [0]:
# prepare dataset either for classification or deep metric learning
import tensorflow_datasets as tfds

dataset_name='cifar10'

dataset_dir=os.path.join(source_path,dataset_name)
if not os.path.isdir(dataset_dir):
  os.mkdir(dataset_dir)

debug=1

if debug:
  split_percent=1
  train_split = tfds.Split.TRAIN.subsplit(tfds.percent[:split_percent])
  test_split = tfds.Split.TEST.subsplit(tfds.percent[:split_percent])
  train_dataset,info = tfds.load(name=dataset_name, split=train_split, as_supervised=True, with_info=True)
  test_dataset,info = tfds.load(name=dataset_name, split=test_split, as_supervised=True, with_info=True)
else:
  train_dataset,info = tfds.load(name=dataset_name, split='train', as_supervised=True, with_info=True)
  test_dataset,info = tfds.load(name=dataset_name, split='test', as_supervised=True, with_info=True)

input_dim=info.features['image'].shape
num_classes=info.features['label'].num_classes

train_mean_path=os.path.join(dataset_dir,'train_mean.npy')
if os.path.exists(train_mean_path):
  train_mean=np.load(train_mean_path)
else:
  train_mean=[]
  num_train_images=info.splits['train'].num_examples
  train_mean=[]
  for example in train_dataset.take(num_train_images):
    image,label=example[0],example[1]
    image=image.numpy().astype('float32')
    if len(train_mean)==0:
      train_mean=image
    else:
      train_mean = train_mean+image

  train_mean=train_mean*1.0/num_train_images
  np.save(train_mean_path,train_mean)








In [0]:
# prepare dataset either for classification or deep metric learning

def _normalize_img(img, label):
    img = img - train_mean
    img = tf.cast(img, tf.float32) / 255.
    return (img, label)


# preprocessing of labels for classification

def _encode_one_hot(img, label):
    label = tf.one_hot(label,num_classes)
    return (img, label)


train_dataset = train_dataset.map(_normalize_img)
test_dataset = test_dataset.map(_normalize_img)

loss_type='triplet'
if loss_type=='classification':
  train_dataset = train_dataset.map(_encode_one_hot)
  test_dataset = test_dataset.map(_encode_one_hot)

# Build your input pipelines

train_dataset = train_dataset.shuffle(1024).batch(32)
test_dataset = test_dataset.batch(32)

In [0]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

# based on the lifted scheme paper
def create_deep_base_network(input_dim,loss_type,num_classes=0,transfer=False,freeze_weights=False):
  weights='imagenet' if transfer else None
  conv_base = ResNet50(weights=weights, include_top=False, input_shape=input_dim)

  if freeze_weights:
    for layer in conv_base.layers:
      layer.trainable=False
  
  model = models.Sequential()
  model.add(layers.Input(input_dim))
  model.add(conv_base)

  model.add(layers.Flatten())
  model.add(layers.BatchNormalization())

  if loss_type=='classification':
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(10, activation='softmax'))
  else:
    model.add(layers.Dense(64, activation=None))
    model.add(layers.Dropout(0.5))

  return model


def create_shallow_network(input_dim,loss_type,num_classes=0):
  model=tf.keras.Sequential()
  model.add(layers.Input(input_dim))
  model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=5, padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
  model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
  model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
  model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, padding='same'))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.BatchNormalization())

  if loss_type=='classification':
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers(Dense(num_classes,activation='softmax')))
  else:
    model.add(tf.keras.layers.Dense(256, activation=None))
    model.add(tf.keras.layers.Dropout(0.5))

  return model

def construct_model(model_type,input_dim,loss_type,num_classes,transfer=False,freeze_weights=False):
  if model_type=='shallow':
    model=create_shallow_network(input_dim,loss_type,num_classes)
  elif model_type=='deep':
    model=create_deep_base_network(input_dim,loss_type,num_classes,transfer,freeze_weights)

  if loss_type =='triplet':
    model.add(tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)))
  return model

def construct_loss(loss_type,margin):
  if loss_type=='triplet':
    loss=tfa.losses.TripletSemiHardLoss(margin=margin)
  elif loss_type=='lifted':
    loss=tfa.losses.LiftedStructLoss(margin=margin)
  elif loss_type=='classification':
    loss=tf.keras.losses.CategoricalCrossentropy()
  return loss

In [19]:
# tune lr

lrs=[1e-1,1e-2,1e-3,1e-4,1e-5]

model_type='deep'
transfer=True
freeze_weights=False

margin=1.0

model_dir=os.path.join(dataset_dir,model_type)
if not os.path.isdir(model_dir):
  os.mkdir(model_dir)

num_epochs=30

fig,ax=plt.subplots(2,3)
ax=ax.ravel()
for lr_index,lr in enumerate(lrs):

  model=construct_model(model_type,input_dim,loss_type,num_classes,transfer,freeze_weights)
  loss=construct_loss(loss_type,margin)

  # Compile the model
  model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss=loss)

  history = model.fit(
      train_dataset,
      epochs=num_epochs)

  ax[lr_index].set_title('Loss for lr_%s'%(str(lr)))
  ax[lr_index].plot(np.arange(num_epochs),history.history['loss'],'r',label='train_loss_lr_%s'%(str(lr)))
  
plt.legend()
plt.savefig(os.path.join(model_dir,'loss_plot_lr_tune_%s_transfer_%s_freeze_%s_%s_%s.png'%(model_type,transfer,freeze_weights,loss_type,margin)))
plt.close()

Epoch 1/30
16/16 [==============================] - 9s 589ms/step - loss: 0.9433
Epoch 2/30
16/16 [==============================] - 2s 114ms/step - loss: 0.9444
Epoch 3/30
16/16 [==============================] - 2s 113ms/step - loss: 0.9243
Epoch 4/30
16/16 [==============================] - 2s 112ms/step - loss: 0.9160
Epoch 5/30
16/16 [==============================] - 2s 112ms/step - loss: 0.9134
Epoch 6/30
16/16 [==============================] - 2s 112ms/step - loss: 0.9122
Epoch 7/30
16/16 [==============================] - 2s 113ms/step - loss: 0.9082
Epoch 8/30
16/16 [==============================] - 2s 112ms/step - loss: 0.9026
Epoch 9/30
16/16 [==============================] - 2s 113ms/step - loss: 0.8967
Epoch 10/30
16/16 [==============================] - 2s 115ms/step - loss: 0.9393
Epoch 11/30
16/16 [==============================] - 2s 113ms/step - loss: 0.9532
Epoch 12/30
16/16 [==============================] - 2s 112ms/step - loss: 0.9403
Epoch 13/30
16/16 [======

In [7]:
# train model

lr=1e-4

model_type='deep'
transfer=True
freeze_weights=False

margin=1.0

model_dir=os.path.join(dataset_dir,model_type)
if not os.path.isdir(model_dir):
  os.mkdir(model_dir)

num_epochs=30

model=construct_model(model_type,input_dim,loss_type,num_classes,transfer,freeze_weights)
loss=construct_loss(loss_type,margin)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss=loss)

# Prepare callbacks for model saving and for learning rate adjustment.
filepath=os.path.join(model_dir,'final_%s_%s_transfer_%s_freeze_%s_%s_%s.h5'%(loss_type,model_type,transfer,freeze_weights,margin))
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)

callbacks = [checkpoint]

history = model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=test_dataset,
    callbacks=callbacks)

plt.figure()
plt.plot(np.arange(num_epochs),history.history['loss'],'r',label='train_loss')
plt.plot(np.arange(num_epochs),history.history['val_loss'],'b',label='val_loss')
plt.legend()
plt.savefig(os.path.join(model_dir,'final_loss_plot%s_transfer_%s_freeze_%s_%s_%s.png'%(model_type,transfer,freeze_weights,loss_type,margin)))
plt.close()

Epoch 1/30
     16/Unknown - 9s 590ms/step - loss: 2.8915
Epoch 00001: val_loss improved from inf to 3.50534, saving model to drive/My Drive/Colab Notebooks/cifar10/deep/final_deep_transfer_True_freeze_False_classification_1.0.h5
16/16 [==============================] - 12s 767ms/step - loss: 2.8915 - val_loss: 0.0000e+00
Epoch 2/30
3/4 [=====================>........] - ETA: 0s - loss: 2.7934
Epoch 00002: val_loss did not improve from 3.50534
16/16 [==============================] - 2s 139ms/step - loss: 2.6828 - val_loss: 3.6797
Epoch 3/30
3/4 [=====================>........] - ETA: 0s - loss: 2.6248
Epoch 00003: val_loss did not improve from 3.50534
16/16 [==============================] - 2s 146ms/step - loss: 2.5172 - val_loss: 4.4345
Epoch 4/30
3/4 [=====================>........] - ETA: 0s - loss: 2.1333
Epoch 00004: val_loss did not improve from 3.50534
16/16 [==============================] - 2s 141ms/step - loss: 2.1864 - val_loss: 4.4976
Epoch 5/30
3/4 [=====================

In [39]:
# tune margin

lr=1e-3
margins=[0.2,1]

model_type='deep'
classification=False
transfer=True
freeze_weights=False

loss_type='triplet'
l2_normalize=1 if loss_type=='triplet' else 0

model_dir=os.path.join(dataset_dir,model_type)
if not os.path.isdir(model_dir):
  os.mkdir(model_dir)

num_epochs=25

fig,ax=plt.subplots(1,len(margins))
ax=ax.ravel()
for margin_index,margin in enumerate(margins):

  model=construct_model(model_type,input_dim,classification,num_classes,transfer,freeze_weights,l2_normalize)
  loss=construct_loss(loss_type,margin)

  # Compile the model
  model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss=loss)

  history = model.fit(
      train_dataset,
      epochs=num_epochs)

  ax[margin_index].set_title('Loss for margin_%s'%(str(margin)))
  ax[margin_index].plot(np.arange(num_epochs),history.history['loss'],'r',label='train_loss_margin_%s'%(str(margin)))
  
plt.legend()
plt.savefig(os.path.join(model_dir,'loss_plot_margin_%s_transfer_%s_freeze_%s_%s_%s.png'%(model_type,transfer,freeze_weights,loss_type,margin)))
plt.close()

Epoch 1/25
16/16 [==============================] - 10s 650ms/step - loss: 0.1618
Epoch 2/25
16/16 [==============================] - 2s 123ms/step - loss: 0.1519
Epoch 3/25
16/16 [==============================] - 2s 127ms/step - loss: 0.1404
Epoch 4/25
16/16 [==============================] - 2s 126ms/step - loss: 0.1343
Epoch 5/25
16/16 [==============================] - 2s 125ms/step - loss: 0.1330
Epoch 6/25
16/16 [==============================] - 2s 126ms/step - loss: 0.1303
Epoch 7/25
16/16 [==============================] - 2s 124ms/step - loss: 0.1263
Epoch 8/25
16/16 [==============================] - 2s 126ms/step - loss: 0.1279
Epoch 9/25
16/16 [==============================] - 2s 126ms/step - loss: 0.1290
Epoch 10/25
16/16 [==============================] - 2s 126ms/step - loss: 0.1274
Epoch 11/25
16/16 [==============================] - 2s 126ms/step - loss: 0.1227
Epoch 12/25
16/16 [==============================] - 2s 126ms/step - loss: 0.1183
Epoch 13/25
16/16 [=====